Code to extract numerical and time series data.

Things to note:

Because the chartevents data is very large:

1. Only 20,000,000 rows were loaded and among these rows
2. For each hadm_id only the first reading for each item was extracted

In [51]:
# Import libraries
import os
import pandas as pd

In [52]:
# Load the data

#items 
items = pd.read_csv(os.path.join('..','d_items.csv'), usecols= ['itemid', 'label', 'abbreviation'])

items = items[items['itemid'].isin([220210, 220277, 225309, 220045, 220739, 223900, 223901, 223762])]

items

,itemid,label,abbreviation
2,220045,Heart Rate,HR
28,220210,Respiratory Rate,RR
36,220277,O2 saturation pulseoxymetry,SpO2
159,220739,GCS - Eye Opening,Eye Opening
338,223762,Temperature Celsius,Temperature C
407,223900,GCS - Verbal Response,Verbal Response
408,223901,GCS - Motor Response,Motor Response
1212,225309,ART BP Systolic,ART BP Systolic


In [53]:
# chartevents
chart = pd.read_csv(os.path.join('..','chartevents.csv'), usecols= ['subject_id', 'hadm_id', 'charttime','itemid', 'valuenum', 'valueuom'], nrows=20000000)

chart = chart[chart['itemid'].isin([220210, 220277, 225309, 220045, 220739, 223900, 223901, 223762])]

chart = chart.groupby(['hadm_id', 'itemid']).first().reset_index()

chart.head()

,hadm_id,itemid,subject_id,charttime,valuenum,valueuom
0,20001770,220045,10117812,2117-01-28 07:00:00,108.0,bpm
1,20001770,220210,10117812,2117-01-28 07:00:00,14.0,insp/min
2,20001770,220277,10117812,2117-01-28 07:00:00,94.0,%
3,20001770,220739,10117812,2117-01-28 08:00:00,3.0,None
4,20001770,223900,10117812,2117-01-28 08:00:00,5.0,None


In [54]:
# Combining the dataset
num_data = pd.merge(chart, items, on = ['itemid'])

num_data.head()

,hadm_id,itemid,subject_id,charttime,valuenum,valueuom,label,abbreviation
0,20001770,220045,10117812,2117-01-28 07:00:00,108.0,bpm,Heart Rate,HR
1,20003008,220045,10289011,2134-06-29 22:38:00,78.0,bpm,Heart Rate,HR
2,20004004,220045,10236621,2171-06-28 19:00:00,99.0,bpm,Heart Rate,HR
3,20005763,220045,10628534,2175-05-31 19:00:00,81.0,bpm,Heart Rate,HR
4,20008400,220045,10559183,2116-01-04 07:40:00,76.0,bpm,Heart Rate,HR


In [55]:
# Use the items as columns
num_data = num_data.pivot_table(index= ['subject_id', 'hadm_id', 'charttime'], columns = 'label', values = 'valuenum').reset_index()

num_data.head(10)

label,subject_id,hadm_id,charttime,ART BP Systolic,GCS - Eye Opening,GCS - Motor Response,GCS - Verbal Response,Heart Rate,O2 saturation pulseoxymetry,Respiratory Rate,Temperature Celsius
0,10000032,29079034,2180-07-23 20:00:00,NaN,4.0,6.0,5.0,NaN,NaN,NaN,NaN
1,10000032,29079034,2180-07-23 22:00:00,NaN,NaN,NaN,NaN,94.0,95.0,20.0,NaN
2,10000980,26913865,2189-06-27 08:54:00,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN
3,10000980,26913865,2189-06-27 08:56:00,NaN,NaN,NaN,NaN,77.0,100.0,NaN,NaN
4,10000980,26913865,2189-06-27 09:09:00,NaN,4.0,6.0,5.0,NaN,NaN,NaN,NaN
5,10001217,24597018,2157-11-21 19:00:00,NaN,NaN,NaN,NaN,101.0,96.0,25.0,NaN
6,10001217,24597018,2157-11-21 20:00:00,NaN,4.0,6.0,5.0,NaN,NaN,NaN,NaN
7,10001217,27703517,2157-12-19 20:00:00,NaN,4.0,6.0,5.0,79.0,92.0,18.0,NaN
8,10001725,25563031,2110-04-12 19:00:00,NaN,NaN,NaN,NaN,82.0,99.0,22.0,NaN
9,10001725,25563031,2110-04-12 20:00:00,NaN,4.0,6.0,5.0,NaN,NaN,NaN,NaN


In [56]:
# Combine all the GCS to get the total
num_data['GCS Total'] = num_data[['GCS - Eye Opening', 'GCS - Motor Response', 'GCS - Verbal Response']].sum(axis=1)

num_data = num_data.drop(columns = ['GCS - Eye Opening', 'GCS - Motor Response', 'GCS - Verbal Response'])

num_data.sample(10)

label,subject_id,hadm_id,charttime,ART BP Systolic,Heart Rate,O2 saturation pulseoxymetry,Respiratory Rate,Temperature Celsius,GCS Total
2329,10160781,24512311,2176-05-11 13:00:00,NaN,96.0,92.0,22.0,NaN,0.0
6743,10454165,23544491,2150-08-05 20:00:00,NaN,90.0,93.0,13.0,NaN,15.0
5078,10331854,21008158,2161-08-04 15:24:00,NaN,NaN,NaN,NaN,NaN,15.0
1168,10082986,27812341,2190-02-27 21:36:00,NaN,131.0,94.0,20.0,NaN,0.0
4285,10287015,27614346,2170-02-10 07:00:00,NaN,87.0,100.0,13.0,37.8,0.0
1315,10096420,25396519,2204-07-16 21:39:00,NaN,NaN,NaN,NaN,NaN,15.0
7758,10522319,20271144,2192-09-04 16:00:00,NaN,77.0,93.0,18.0,NaN,15.0
7408,10501066,26574121,2122-11-24 08:00:00,NaN,103.0,100.0,27.0,NaN,8.0
3301,10225619,21605488,2129-05-10 07:00:00,NaN,NaN,NaN,NaN,38.2,0.0
9107,10607649,27071401,2157-08-22 02:00:00,NaN,NaN,94.0,NaN,NaN,0.0


In [57]:
# Get the age column
patient_data = pd.read_csv(os.path.join('..','patients.csv', 'patients.csv'), usecols= ['subject_id','anchor_age'])

patient_data.head()

,subject_id,anchor_age
0,10000032,52
1,10000048,23
2,10000068,19
3,10000084,72
4,10000102,27


In [58]:
# Merge the age column to the numerical dataset
final = pd.merge(patient_data, num_data, on = ['subject_id'])

final.head()

,subject_id,anchor_age,hadm_id,charttime,ART BP Systolic,Heart Rate,O2 saturation pulseoxymetry,Respiratory Rate,Temperature Celsius,GCS Total
0,10000032,52,29079034,2180-07-23 20:00:00,NaN,NaN,NaN,NaN,NaN,15.0
1,10000032,52,29079034,2180-07-23 22:00:00,NaN,94.0,95.0,20.0,NaN,0.0
2,10000980,73,26913865,2189-06-27 08:54:00,NaN,NaN,NaN,23.0,NaN,0.0
3,10000980,73,26913865,2189-06-27 08:56:00,NaN,77.0,100.0,NaN,NaN,0.0
4,10000980,73,26913865,2189-06-27 09:09:00,NaN,NaN,NaN,NaN,NaN,15.0


In [62]:
final.to_csv('NumericalData.csv')